# RSA Encryption

Below are two Python functions that may be used for generating large prime numbers.

In [83]:
import math
import numpy as np
from random import randint

def is_prime(num, test_count):
    # Use a sample of test_count numbers to check if num is prime.
    # The algorithm uses the (probabilistic) Fermat primality test.
    
    # Check the obvious cases.
    if num == 1:
        return False
    if num % 2 == 0:
        return False
    
    # There's no point testing more numbers than are possible.
    if test_count >= num:
        test_count = num - 1
        
    for x in range(test_count):
        val = randint(1, num-1)
        # val and num are co-prime if val**(num-1) % num = 1.
        # (Fermat's little theorem)
        if pow(val, num-1, num) != 1:
            # Exit function and return False if a factor is found.
            return False
    
    # If no factors are found, assume that num is prime.
    # (Note that num might not actually be prime!)
    return True

def generate_big_prime(n, test_count=1):
    # returns a random prime number of length n bits
    
    found_prime = False
    while not found_prime:
        p = randint(2**(n-1), 2**n)
        # Check test_count samples to see if p is prime.
        if is_prime(p, test_count):
            return p
        
generate_big_prime(256)

75017021989362269278564062821115446890627444684398270294505565830410946417921

Write some Python code below that generates a public key (e, N) and private key (d, N) as described in Lecture 13.  Choose for N an 8-bit integer.

In [84]:
# Put your code here.

#need to check 7d mod M and 1 mod M, where M is the totient. 7D / mod == 1. 7D can be any multiple of totient
#think of it as 7D mod totient ==1 to make it easier.
# Choose p and q
#want to generate a 4 bit prime for p and q bc 4 *2 = 8 bit M
#want n to be the same for both p and q 
n = 8
p=0
q=0
while  p == q:
    p = generate_big_prime(n)
    q = generate_big_prime(n)
print('q is',q)
print('p is',p)
# Compute e
#co-prime number is a prime number and non-prime number or 2 non-prime numbers; any 2 prime numbers are co-prime numbers
M = (q-1)*(p-1) #totient
print('M is', M)
N= q*p


e = randint(1, M)
g = math.gcd(e, M)
while g != 1:
        e= randint(1,M)
        g = math.gcd(e, M)
print('e is',e)
# Compute d
d=0
k=1
con_fac=int((1+ (k*M))/e)
while (d*e)% M != 1  or (1+ (k*M))%e != 0:
    k+=1
    d=(1+ (k*M))/e
    
print('N is', N)    
print('d is', d)
print('k is', k)
    
    


print("Prime Factorization: " + str(N) + " = " + str(p) + " * " + str(q))
print("Public Key: " + "(" + str(e) + ", " + str(N) + ")")
print("Private Key: " + "(" + str(d) + ", " + str(N) + ")")

q is 137
p is 151
M is 20400
e is 14233
N is 20687
d is 4297.0
k is 2998
Prime Factorization: 20687 = 151 * 137
Public Key: (14233, 20687)
Private Key: (4297.0, 20687)


Write some Python code that generates a random integer message x < N.
Use the public key from above to construct the encrypted message y.

In [85]:
# Put your code here.
publickey = (e, N)
x=randint(1, N)
y=pow(x, e, N)

print("Unencrypted Message: x = " + str(x))
print("Encrypted Message: y = " + str(y))

Unencrypted Message: x = 9252
Encrypted Message: y = 13348


Now, use the private key to decrypt the encrypted message y.  Do you recover the original message?

In [86]:
# Put your code here.
x_decrypted = pow(y, int(d), N)
print("Decrypted Message: " + str(x_decrypted))

Decrypted Message: 9252


Write a function to determine the prime factors of N using a brute-force search.  Use this to infer the private key.  Does it correctly decrypt the message y?

In [88]:
def brute_force_factor(N):
    p=2
    q=int(N/2)
    search_space = int(math.sqrt(N)) + 1
    for i in range(3, search_space,2):
        if N % i == 0 and is_prime(i,1):
            p= i
            q= int(N/i)
            return p,q
            
    return p,q
      
p_hacked, q_hacked = brute_force_factor(N)
print('values are',p_hacked, q_hacked)
# Compute d_hacked (Note: It could be different from the one above.)
M_h=(p_hacked-1)*(q_hacked-1)
#use the same e, which is given in the public key
print('e is',e)

# Compute d
d_hacked = 0
k_h = 1
while (d_hacked*e)% M_h != 1 or (1+ (k_h*M_h))%e != 0:
    k_h+=1
    d_hacked= int((1+ (k_h*M_h))/e)
    

x_hacked=pow(y, int(d), N)

print("Hacked Factorization: " + str(N) + " = " + str(p_hacked) + " * " + str(q_hacked))
print("Hacked Private Key: " + "(" + str(d_hacked) + ", " + str(N) + ")")
print("Hacked Message: " + str(x_hacked))

values are 137 151
e is 14233
Hacked Factorization: 20687 = 137 * 151
Hacked Private Key: (4297, 20687)
Hacked Message: 9252


I assume most of the time it does, at least  for all the cases I've run. Its because we have restricted the search space to sqrt(N) and we take advantage of the fact that N is a factor of two prime numbers which should be the only factors always.If we did not know this beforhand, then we would have to do another approach.

Share your public key and encrypted message with someone else.  Can you hack their encryption?

Public Key: (16673, 22663)
from brute force:
values are 131 173
e is 16673
Hacked Factorization: 22663 = 131 * 173
Hacked Private Key: (2017, 22663)
Hacked Message: 13367
Private Key: (2017, 22663)